In [2]:
import numpy as np
import pandas as pd
import requests
import json
import dotenv
import os

## Getting Biographical Data from the Congress API

The most important skill is reading the API documentation efficiently.

Look ONLY for these five things:

1. Root: The beginning part of all URLs associated with this API for getting data back (I often find this in examples first)

https://api.congress.gov/v3

2. Endpoint: the second part of the URL, generally it points to a specific data set to be returned. 

/member/{bioguideId}

The curly braces mean fill this in for whicver member of congress you want. 

3. The headers: data that WE give to the API to identify ourselves, and provide other information if requested. 
 * User Agent string: a string that identifies your software. If you are writing a Bot, what is your bots name, and what methods are you using to access this API.

 The standard convention is:

 'botname/version (email) python-requests/request-package-version'

In [ ]:
import requests

botname = 'targ'
version = '0.0'
email = 'ppj8hm@virginia.edu'

useragent = f'{botname}/{version} ({email}) python-requests/{requests.__version__}'
headers = {'User-Agent': useragent}
headers


{'User-Agent': 'targ/0.0 (ppj8hm@virginia.edu) python-requests/2.32.5'}

: 

: 

: 

: 

4. API Parameters: these are ways to control what data is requested. The parameters are generally listed in the documentation for each endpoint. 3 kinds of parameter:

* Query parameters: should be listed in a dictionary and passed to the `params` argument of `requests.get()`

* Path parameters: are part of the endpoint (such as bioguideId)
  
* Global parameters: apply to any use of the API regardless of endpoint. Often that includes your API key. Can be in either the headers or the params.

5. If an API Key is needed, how to get one, and how will you let the API know the key?

API keys should be saved in a .env file in the same folder as your code. Use `dotenv` package to load the key into your code without ever displaying/exposing your key. 

In [ ]:
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')

: 

: 

: 

: 

In [ ]:
params = { 'format': 'json',
    'api_key': congresskey
}



: 

: 

: 

: 

Put it all together:

In [ ]:
import requests

bioguideID = 'M001239'
root = 'https://api.congress.gov/v3'
endpoint = f'/member/{bioguideID}'

r = requests.get(
    root + endpoint,
    headers=headers,
    params=params)
r

<Response [200]>

: 

: 

: 

: 

In [ ]:
r.json()

{'member': {'addressInformation': {'city': 'Washington',
   'district': 'DC',
   'officeAddress': '1013 Longworth House Office Building',
   'phoneNumber': '(202) 225-4711',
   'zipCode': 20515},
  'bioguideId': 'M001239',
  'birthYear': '1968',
  'cosponsoredLegislation': {'count': 174,
   'url': 'https://api.congress.gov/v3/member/M001239/cosponsored-legislation'},
  'currentMember': True,
  'depiction': {'attribution': 'Image courtesy of the Member',
   'imageUrl': 'https://www.congress.gov/img/member/67744ba20b34857ecc909149_200.jpg'},
  'directOrderName': 'John J. McGuire III',
  'district': 5,
  'firstName': 'John',
  'honorificName': 'Mr.',
  'invertedOrderName': 'McGuire, John J.',
  'lastName': 'McGuire',
  'middleName': 'J.',
  'officialWebsiteUrl': 'https://mcguire.house.gov/',
  'partyHistory': [{'partyAbbreviation': 'R',
    'partyName': 'Republican',
    'startYear': 2025}],
  'previousNames': [{'directOrderName': 'John McGuire',
    'endDate': '2025-01-10T17:36:18Z',
   

: 

: 

: 

: 

## Sponsored Legislation

In [ ]:
endpoint = f'/member/{bioguideID}/sponsored-legislation'
endpoint

'/member/M001239/sponsored-legislation'

: 

: 

: 

: 

In [ ]:
params = { 'format': 'json',
          'offset': 0,
          'limit': 250,
          'api_key': congresskey}

: 

: 

: 

: 

In [ ]:
r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)
r

<Response [200]>

: 

: 

: 

: 

In [ ]:
myjson = r.json()

: 

: 

: 

: 

In [ ]:
myjson['sponsoredLegislation']

sponsoredLegislation_df =  pd.json_normalize(myjson, record_path=['sponsoredLegislation'])

: 

: 

: 

: 

In [ ]:
for x in sponsoredLegislation_df['title']:
    print(x)

Requiring Excise for Migrant Income Transfers Act” or the “REMIT Act.
Peace Through Strength Act of 2025
Border Wall Status Act
Make the District of Columbia Safe and Beautiful Act
FAIR Act
VA Data Transparency and Trust Act
Manned Aircraft Clarification Act
Uranium for Energy Independence Act of 2025
Agricultural and Forestry Hauling Efficiency Act


: 

: 

: 

: 

In [ ]:
r = requests.get(sponsoredLegislation_df['url'][3],
                    headers=headers,
                    params=params)
r

<Response [200]>

: 

: 

: 

: 

In [ ]:
r.json()

{'bill': {'actions': {'count': 6,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/actions?format=json'},
  'cboCostEstimates': [{'description': 'As ordered reported by the House Committee on Oversight and Government Reform on \nSeptember 10, 2025\n',
    'pubDate': '2025-10-03T14:03:00Z',
    'title': 'H.R. 5103, Make the District of Columbia Safe and Beautiful Act of 2025',
    'url': 'https://www.cbo.gov/publication/61792'}],
  'committees': {'count': 2,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/committees?format=json'},
  'congress': 119,
  'constitutionalAuthorityStatementText': '<pre>\n[Congressional Record Volume 171, Number 144 (Wednesday, September 3, 2025)]\n[House]\nFrom the Congressional Record Online through the Government Publishing Office [<a href="https://www.gpo.gov">www.gpo.gov</a>]\nBy Mr. McGUIRE:\nH.R. 5103.\nCongress has the power to enact this legislation pursuant\nto the following:\nArticle I, Section 8\n[Page H3829]\n</pre>',
  'cosponsors'

: 

: 

: 

: 

## FEC Data for financial campagin contributions

In [ ]:
dotenv.load_dotenv()
feckey = os.getenv('feckey')

: 

: 

: 

: 

In [ ]:

root = 'https://api.open.fec.gov/'
endpoint = '/v1/candidates/search/'

params = {'api_key': feckey,
          'q':'John McGuire',
           'state': 'VA',
           'district': '5',
           'office': 'H',
           'year': '2024'}
r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)
r


<Response [200]>

: 

: 

: 

: 

In [ ]:
myjson = r.json()
candidate_id = myjson['results'][0]['candidate_id']
candidate_id 

'H0VA07133'

: 

: 

: 

: 

In [ ]:
endpoint = '/v1/schedules/schedule_a/by_size/by_candidate/'
params = { 'api_key': feckey,
           'candidate_id': candidate_id,
           'cycle' : 2024}

r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)
r

<Response [200]>

: 

: 

: 

: 

In [ ]:
r.json()

{'api_version': '1.0',
 'pagination': {'count': 0,
  'is_count_exact': True,
  'page': 1,
  'pages': 0,
  'per_page': 20},
 'results': []}

: 

: 

: 

: 

In [ ]:
endpoints = f'/v1/candidate/{candidate_id}/committee/'
params = { 'api_key' : feckey,
              'cycle' : 2024}

r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)
r
r.json()

{'message': 'Please choose a single `two_year_transaction_period` or add one of the following filters to your query: `committee_id`, `contributor_id`, `contributor_name`, `contributor_city`, `contributor_zip`, `contributor_employer`, `contributor_occupation`, `image_number`',
 'status': 400}

: 

: 

: 

: 

In [ ]:
committee_ids = [item['committee_id'] for item in r.json()['results']]

KeyError: 'results'

: 

: 

: 

: 

In [ ]:
endpoint = '/v1/schedules/schedule_a/'
params = { 'api_key': feckey,
           'committee_id': committee_ids[0],
           'per_page': 100,
           'sort':'-contributor_receipt_amount'}

r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)

NameError: name 'committee_ids' is not defined

: 

: 

: 

: 

In [ ]:
contrib_dict = [{'contributeor_name': x['contributor_name'],
                 'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                 'memo_text': x['memo_text'],
                 'pdf_url': x['pdf_url']} for x in r.json()['results']]
pd.DataFrame(contrib_dict)
contrib_dict = pd.DataFrame(contrib_dict)
contrib_dict

KeyError: 'results'

: 

: 

: 

: 

In [ ]:
lastindex = r.json()['pagination']['last_indexes']

KeyError: 'pagination'

: 

: 

: 

: 

In [ ]:
lastindex

: 

: 

: 

: 

In [ ]:
params['last_contribution_receipt_amount'] = lastindex['last_contribution_receipt_amount']
params['last_index'] = lastindex['last_index']

: 

: 

: 

: 

In [ ]:
r = requests.get(root + endpoint,
                 headers=headers,
                 params=params) 
r

: 

: 

: 

: 

## Loop over all contributions to McGuire

In [ ]:
contrib_df = pd.DataFrame()

endpoint = '/v1/schedules/schedule_a/'
params = { 'api_key': feckey,
           'committee_id': committee_ids[0],
           'per_page': 100,
           'sort':'-contributor_receipt_amount'}

r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)

contrib_list = [{'contributeor_name': x['contributor_name'],
                 'contributor_aggregate_ytd': x['contributor_aggregate_ytd'],
                 'memo_text': x['memo_text'],
                 'pdf_url': x['pdf_url']} for x in r.json()['results']]
lastindex = r.json()['pagination']['last_indexes']

#newrecord =len(r.json()['results'].keys())

#contrib_df_list = [pd.DataFrame(contrib_dict)]


newrecord =len(r.json()['results'])

while newrecord > 0:
    print(contrib_df.shape[0])
    
    params['last_contribution_receipt_amount'] = lastindex['last_contribution_receipt_amount']
    params['last_index'] = lastindex['last_index']

    r = requests.get(root + endpoint,
                     headers=headers,
                     params=params)
    
    #contrib_df = pd.concat{[contrib_df, pd.DataFrame(contrib_dict)], ignore_index=True}
    #lastindex = r.json()['pagination']['last_indexes']



: 

: 

: 

: 